# Area for testing Code

### Try code loading weather from weather station

In [6]:
import requests
import csv
import lxml.html as lh

from util.UnitConverter import ConvertToSystem
from util.Parser import Parser
from util.Utils import Utils
from datetime import date

default_station = "https://www.wunderground.com/dashboard/pws/IDRSING3"

def scrap_station(weather_station_url="https://www.wunderground.com/dashboard/pws/IDRSING3", 
                  unit_system="metric", start_DATE="", end_DATE="", timeout = 5):
    """Get date from Weather Station. Create temporary .csv and return 
    path to it."""
    
    session = requests.Session()
    
    url_gen = Utils.date_url_generator(weather_station_url, start_DATE, end_DATE)
    station_name = weather_station_url.split('/')[-1]
    file_name = f'{station_name}.csv'

    with open(file_name, 'a+', newline='') as csvfile:
        fieldnames = ['Date', 'Time',	'Temperature',	'Dew_Point',	'Humidity',	'Wind',	'Speed',	'Gust',	'Pressure',	'Precip_Rate',	'Precip_Accum',	'UV',   'Solar']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write the correct headers to the CSV file
        # 12:04 AM	24.4 C	18.3 C	69 %	SW	0.0 km/h	0.0 km/h	1,013.88 hPa	0.00 mm	0.00 mm	0	0 w/m²
        writer.writerow({'Date': 'Date', 'Time': 'Time', 
                         'Temperature': 'Temperature_C', 'Dew_Point':
                         'Dew_Point_C',	'Humidity': 'Humidity_%', 
                         'Wind': 'Wind', 'Speed': 'Speed_kmh',	
                         'Gust': 'Gust_kmh',	'Pressure': 'Pressure_hPa',	
                         'Precip_Rate': 'Precip_Rate_mm',	
                         'Precip_Accum': 'Precip_Accum_mm',	'UV': 'UV',   'Solar': 'Solar_w/m2'})

        for date_string, url in url_gen:
            try:
                print(f'Weather data from {url}')
                history_table = False
                while not history_table:
                    html_string = session.get(url, timeout=timeout)
                    doc = lh.fromstring(html_string.content)
                    history_table = doc.xpath('//*[@id="main-page-content"]/div/div/div/lib-history/div[2]/lib-history-table/div/div/div/table/tbody')
                    if not history_table:
                        print("refreshing session")
                        session = requests.Session()

                # parse html table rows
                data_rows = Parser.parse_html_table(date_string, history_table)

                # convert to metric system
                converter = ConvertToSystem(unit_system)
                data_to_write = converter.clean_and_convert(data_rows)
                    
                print(f'Saving {len(data_to_write)} rows')
                writer.writerows(data_to_write)
            except Exception as e:
                print(e)

In [7]:
### Test running the scrapper

d1 = date(2021, 2, 1)
d2 = date(2021, 2, 15)

scrap_station(start_DATE=d1, end_DATE=d2)

Weather data from https://www.wunderground.com/dashboard/pws/IDRSING3/table/2021-02-01/2021-02-01/daily
HTTPSConnectionPool(host='www.wunderground.com', port=443): Read timed out. (read timeout=5)
Weather data from https://www.wunderground.com/dashboard/pws/IDRSING3/table/2021-02-02/2021-02-02/daily
Saving 287 rows
Weather data from https://www.wunderground.com/dashboard/pws/IDRSING3/table/2021-02-03/2021-02-03/daily
Saving 288 rows
Weather data from https://www.wunderground.com/dashboard/pws/IDRSING3/table/2021-02-04/2021-02-04/daily
Saving 288 rows
Weather data from https://www.wunderground.com/dashboard/pws/IDRSING3/table/2021-02-05/2021-02-05/daily
Saving 288 rows
Weather data from https://www.wunderground.com/dashboard/pws/IDRSING3/table/2021-02-06/2021-02-06/daily
Saving 288 rows
Weather data from https://www.wunderground.com/dashboard/pws/IDRSING3/table/2021-02-07/2021-02-07/daily
Saving 272 rows
Weather data from https://www.wunderground.com/dashboard/pws/IDRSING3/table/2021-02